In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.io.parsers.read_csv("train_4aqQp50.csv")

In [ ]:
test = pd.io.parsers.read_csv("test_VJP2kVH.csv")

In [ ]:
y = train.SALES_PRICE.ravel()
train= train.drop('SALES_PRICE',axis=1)

In [ ]:
i1 = pd.DataFrame(test.PRT_ID)

In [ ]:
combined_set = pd.concat([train, test], axis = 0)

In [ ]:
combined_set = combined_set.drop(['PRT_ID'], axis =1 )

In [ ]:
combined_set['DATE_SALE'] =  pd.to_datetime(combined_set['DATE_SALE'],  format='%d-%m-%Y',errors='coerce')
combined_set['DATE_BUILD'] =  pd.to_datetime(combined_set['DATE_BUILD'], format='%d-%m-%Y',errors='coerce')

In [ ]:
a = combined_set['DATE_BUILD']
b = combined_set['DATE_SALE']
age = b -a
age = age.dt.days
age = pd.DataFrame(age)
age.columns = ['age_of_building']

In [ ]:
combined_set = pd.concat([combined_set, age], axis=1)

In [ ]:
combined_set = combined_set.drop(['DATE_SALE','DATE_BUILD'], axis =1 )

In [ ]:
for feature in combined_set.columns: 
    if combined_set[feature].dtype == 'object': 
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes 

In [ ]:
combined_set.info()

In [ ]:
train = combined_set.iloc[0:7109].values
test = combined_set.iloc[7109:].values

In [ ]:
xgdmat = xgb.DMatrix(train, y)

In [ ]:
test1 = xgb.DMatrix(test)

In [ ]:
our_params = {'eta': 0.005, 'seed':0, 'subsample': 0.6, 'colsample_bytree': 0.5, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1.2, 'nthread':-1} 

In [ ]:
final_gb = xgb.train(our_params, xgdmat, num_boost_round = 50000)

In [ ]:
pred = final_gb.predict(test1)

In [ ]:
i2 = pred
i1.columns = ['PRT_ID']
i2 = pd.DataFrame(i2)
i2.columns = ['SALES_PRICE']

In [ ]:
result = pd.concat([i1, i2], axis=1)

In [ ]:
result.head()

In [ ]:
result.to_csv('pred.csv', index= False)